# Slopes estimation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [266]:
from collections import deque

from rashomon.tva import enumerate_policies, policy_to_profile, enumerate_profiles
from rashomon import loss
from rashomon import counter
# from rashomon import metrics
from rashomon.extract_pools import extract_pools
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache
from rashomon.aggregate import (RAggregate_profile_slopes, RAggregate_slopes,
#     find_profile_lower_bound, find_feasible_combinations, remove_unused_poolings, subset_data
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sample data

In [14]:
M = 2
R = np.array([6, 6])

num_profiles = 2**M
profiles, profile_map = enumerate_profiles(M)
all_policies = enumerate_policies(M, R)
num_all_policies = len(all_policies)

profile_on = (1, 1)

policies_on = [pol for pol in all_policies if policy_to_profile(pol) == profile_on]
num_policies_on = len(policies_on)

n_per_pol = 10

num_data = num_all_policies * n_per_pol
X = np.zeros(shape=(num_data, M))
D = np.zeros(shape=(num_data, 1), dtype='int_')
y = np.zeros(shape=(num_data, 1))

beta_1_1 = 4
beta_1_2 = 0

beta_2_1 = -3
beta_2_2 = 5

idx_ctr = 0
for idx, policy in enumerate(all_policies):

    if policy_to_profile(policy) == (0, 0) or policy_to_profile(policy) == (1, 1):
        y_i = 0

    elif policy_to_profile(policy) == (1, 0):
        if policy[0] <= 3:
            y_i = beta_1_1 * policy[0]
        else:
            y_i = beta_1_2 * policy[0]
    elif policy_to_profile(policy) == (0, 1):
        if policy[1] <= 3:
            y_i = beta_2_1 * policy[1]
        else:
            y_i = beta_2_2 * policy[1]

    policy_idx = [i for i, x in enumerate(all_policies) if x == policy]
    
    # pool_id = pi_policies[k][idx]
    # mu_i = mu[k][pool_id]
    # var_i = var[k][pool_id]
    y_i = y_i + np.random.normal(0, 1, size=(n_per_pol, 1))

    start_idx = idx_ctr * n_per_pol
    end_idx = (idx_ctr + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = policy_idx[0]
    y[start_idx:end_idx, ] = y_i

    idx_ctr += 1

In [15]:
X.shape

(360, 2)

In [203]:
policies_subset = [p for p in all_policies if policy_to_profile(p) == (1, 0)]

subset_idx = [i for i, p in enumerate(D[:, 0]) if policy_to_profile(all_policies[p]) == (1, 0)]
X_subset = X[subset_idx, :]
y_subset = y[subset_idx, :]
D_subset = D[subset_idx, :]

policies_temp = [(i, x) for i, x in enumerate(all_policies) if policy_to_profile(x) == (1, 0)]
unzipped_temp = list(zip(*policies_temp))
policy_subset_idx = list(unzipped_temp[0])

# for i, policy_i in enumerate(policies_subset):
#     policy_i = tuple([int(x) for x in X_subset[i, :]])
#     policy_idx = [idx for idx in range(num_all_policies) if all_policies[idx] == policy_i]
#     D_subset[i, 0] = int(policy_idx[0])

range_list = list(np.arange(len(policy_subset_idx)))
policy_map = {i: x for i, x in zip(policy_subset_idx, range_list)}
D_subset = np.vectorize(policy_map.get)(D_subset)

print(X_subset.shape)

(50, 2)


In [38]:
policy_map

{(1, 0): 0, (2, 0): 1, (3, 0): 2, (4, 0): 3, (5, 0): 4}

In [72]:
sigma_true = np.array([[1, 1, 0, 1]], dtype=np.float64)

## RAggregate

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [186]:
R_set = RAggregate_profile_slopes(1, 6, 4, D_subset, X_subset, y_subset, 4,
                                  profile=(1, 0), reg=1, policies=policies_subset, normalize=0)

In [267]:
R_set, R_profiles = RAggregate_slopes(M, R, 10, D, X, y, 7, reg=1, verbose=True)

(0, 0) 6.0693583920274925
1
(0, 1) 6.196114930046413
12
(1, 0) 6.202808014791737
16
(1, 1) 6.673290693389906
95
Finding feasible combinations
Min = 6.999410950369937, Max = 18.903205730405002


In [268]:
for r, model_r in enumerate(R_set):
    for k, prof_k in enumerate(model_r):
        print(profiles[k])
        print(R_profiles[k].sigma[prof_k])

(0, 0)
None
(0, 1)
[[1. 1. 0. 1.]]
(1, 0)
[[1. 1. 0. 1.]]
(1, 1)
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
